In [2]:
import pandas as pd
import numpy as np
from lightgbm import LGBMRegressor
import polars as pl

In [11]:
df = pd.read_csv("/home/shunsuke/OSS/python/CNN_preds.csv")

In [12]:
task1_target = df.loc[:, df.columns.str.startswith("gt_score")].mean(axis=1)
task2_target = df.loc[:, df.columns.str.startswith("gt_ostats")]
df.drop(columns=df.columns[df.columns.str.startswith("gt_")], inplace=True)

In [13]:
task1_target

0        14.000000
1        12.333333
2        16.000000
3        34.000000
4        11.000000
           ...    
15695    10.333333
15696    20.666667
15697    28.666667
15698    23.333333
15699    27.000000
Length: 15700, dtype: float64

In [14]:
a = np.arange(0, 314)
idx = np.arange(0, df.shape[0])
from sklearn.model_selection import train_test_split
train_idx, test_idx = train_test_split(a, test_size=0.2, random_state=42)
# train_idxを、idxのmod314がtrain_idxに含まれるものとして、train_idxを取得
train_idx = [i for i in idx if i % 314 in train_idx]
test_idx = [i for i in idx if i % 314 in test_idx]

train_df = df.iloc[train_idx]
test_df = df.iloc[test_idx]

task1_target_train = task1_target.iloc[train_idx]
task1_target_test = task1_target.iloc[test_idx]
task2_target_train = task2_target.iloc[train_idx]
task2_target_test = task2_target.iloc[test_idx]

In [21]:
from lightgbm import LGBMRegressor
import pickle
import lightgbm as lgb

model = LGBMRegressor(metric="l1")
model.fit(train_df, task1_target_train, eval_set=(test_df, task1_target_test), 
          callbacks=[lgb.early_stopping(100, first_metric_only=True)])

'''with open("model.pkl", "wb") as f:
    pickle.dump(model, f)'''

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.055518 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 51506
[LightGBM] [Info] Number of data points in the train set: 12550, number of used features: 202
[LightGBM] [Info] Start training from score 20.155378
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[49]	valid_0's l1: 4.34365
Evaluated only: l1


'with open("model.pkl", "wb") as f:\n    pickle.dump(model, f)'

In [19]:

pred = model.predict(test_df)
pd.DataFrame({"pred": pred, "gt": task1_target_test})

,pred,gt
3,19.167427,34.000000
5,25.473575,25.333333
7,12.863668,14.666667
9,22.996982,17.666667
17,28.038484,32.000000
...,...,...
15673,12.521010,10.333333
15677,27.251916,25.666667
15678,21.991646,34.333333
15680,23.851515,23.000000


In [77]:
from sklearn.metrics import f1_score
def f1_metric(labels, preds):    
    # F1スコア（weighted平均で計算
    preds = np.argmax(preds, axis=1)
    f1 = f1_score(labels, preds, average='weighted')
    return 'f1', f1, True

def mae_metric(labels, preds):
    preds = np.argmax(preds, axis=1)
    mae = np.mean(np.abs(preds - labels))
    return 'mae', mae, True

In [79]:
from lightgbm import LGBMClassifier
from sklearn.multioutput import MultiOutputClassifier

# Assuming `X_train` is your feature set and `Y_train` is your multi-target dataset with 8 columns.
#model = MultiOutputClassifier(LGBMClassifier())
for col in task2_target_train:
    model = LGBMClassifier(verbose=1)
    train_target, test_target = task2_target_train[col], task2_target_test[col]
    model.fit(train_df, train_target,  eval_set=[(test_df, test_target)], eval_metric = f1_metric,
                        callbacks=[lgb.early_stopping(10, first_metric_only=True)])
    '''with open(f"model2_{col}.pkl", "wb") as f:
        pickle.dump(model, f)'''


'''with open("model2.pkl", "wb") as f:
    pickle.dump(model, f)'''

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.055640 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 51506
[LightGBM] [Info] Number of data points in the train set: 12550, number of used features: 202
[LightGBM] [Info] Start training from score -1.536469
[LightGBM] [Info] Start training from score -1.143426
[LightGBM] [Info] Start training from score -0.890724
[LightGBM] [Info] Start training from score -2.886396
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[10]	valid_0's multi_logloss: 1.34231	valid_0's f1: 0.372042
Evaluated only: multi_logloss
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.060020 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 51506
[LightGBM] [Info] Number of data points in the train set: 12550, number of used features: 202
[LightGBM] [Inf

'with open("model2.pkl", "wb") as f:\n    pickle.dump(model, f)'

In [45]:
model.predict(df).shape

(15700,)

In [46]:
model.predict(df)

array([0, 0, 1, ..., 2, 2, 0])

In [92]:
task2_target

,gt_ostats_respect,gt_ostats_motion,gt_ostats_instrument,gt_ostats_suture,gt_ostats_flow,gt_ostats_knowledge,gt_ostats_performance,gt_ostats_final_quality
0,0,1,0,0,0,1,1,0
1,2,0,0,0,0,0,0,0
2,1,0,0,1,0,1,1,1
3,3,3,3,3,3,3,3,3
4,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
15695,0,0,0,0,0,0,0,0
15696,2,1,1,1,0,2,1,2
15697,2,3,2,2,2,3,2,2
15698,2,1,1,2,1,2,1,2
